In [2]:
#!/usr/bin/env python3

def gf256v_mul_256(a: int, b: int) -> int:
    """
    Multiplies a 256-bit integer 'a' by an 8-bit integer 'b' in GF(2^8),
    where the irreducible polynomial is x^8 + x^4 + x^3 + x + 1 (0x1b).

    Each of the 32 bytes in 'a' is multiplied by the same GF(2^8) element 'b'.

    :param a: 256-bit integer (Python int).
    :param b: 8-bit integer (0 <= b <= 0xFF).
    :return:  256-bit integer result of the GF(2^8) multiply.
    """
    # Mask for the MSB of each byte: 0x80 repeated 32 times for 256 bits
    mask_msb = 0x8080808080808080808080808080808080808080808080808080808080808080

    # Reduce 'a' to 256 bits; reduce 'b' to 8 bits.
    a &= (1 << 256) - 1
    b &= 0xFF

    result = 0

    # We have 8 bits in 'b', so iterate for i in {0, 1, ..., 7}.
    for i in range(8):
        # If the i-th bit of 'b' is set, XOR current 'a' into 'result'.
        if (b >> i) & 1:
            result ^= a

        # --- GF(2^8) reduction step, applied in parallel to all 32 bytes. ---
        # 1) Extract MSB of each byte (bit 7).
        a_msb = a & mask_msb
        # 2) Clear those top bits from 'a'.
        a ^= a_msb
        # 3) Shift left by 1 (multiply by x).
        a <<= 1
        # 4) For each byte that had MSB set, XOR 0x1B (the AES polynomial) into that byte.
        #    Shifting right by 7 moves each byte's MSB into that byte's LSB position in a 256-bit integer.
        #    Multiplying by 0x1B replicates the operation in the correct byte-locations.
        a ^= (a_msb >> 7) * 0x1B

        # Ensure 'a' remains 256 bits.
        a &= (1 << 256) - 1

    # Mask result to 256 bits before returning.
    return result & ((1 << 256) - 1)


def main():
    # Example 256-bit 'a' (32 bytes).
    # For instance: 0x0123456789ABCDEF repeated 2 times, plus more to make 32 bytes total
    a_256 = 0x9ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF012345678
            
    # Example 8-bit 'b'
    
    b_8   = 0x00
    for i in range(0x100):
        product = gf256v_mul_256(a_256, i)
        # print(f"a = 0x{a_256:064x}")
        print(f"b = 0x{i:02x}")
        print(f"w0 = 0x{product:064x}")

if __name__ == "__main__":
    main()


b = 0x00
w0 = 0x0000000000000000000000000000000000000000000000000000000000000000
b = 0x01
w0 = 0x9abcdef0123456789abcdef0123456789abcdef0123456789abcdef012345678
b = 0x02
w0 = 0x2f63a7fb2468acf02f63a7fb2468acf02f63a7fb2468acf02f63a7fb2468acf0
b = 0x03
w0 = 0xb5df790b365cfa88b5df790b365cfa88b5df790b365cfa88b5df790b365cfa88
b = 0x04
w0 = 0x5ec655ed48d043fb5ec655ed48d043fb5ec655ed48d043fb5ec655ed48d043fb
b = 0x05
w0 = 0xc47a8b1d5ae41583c47a8b1d5ae41583c47a8b1d5ae41583c47a8b1d5ae41583
b = 0x06
w0 = 0x71a5f2166cb8ef0b71a5f2166cb8ef0b71a5f2166cb8ef0b71a5f2166cb8ef0b
b = 0x07
w0 = 0xeb192ce67e8cb973eb192ce67e8cb973eb192ce67e8cb973eb192ce67e8cb973
b = 0x08
w0 = 0xbc97aac190bb86edbc97aac190bb86edbc97aac190bb86edbc97aac190bb86ed
b = 0x09
w0 = 0x262b7431828fd095262b7431828fd095262b7431828fd095262b7431828fd095
b = 0x0a
w0 = 0x93f40d3ab4d32a1d93f40d3ab4d32a1d93f40d3ab4d32a1d93f40d3ab4d32a1d
b = 0x0b
w0 = 0x0948d3caa6e77c650948d3caa6e77c650948d3caa6e77c650948d3caa6e77c65
b = 0x0c
w0 = 0xe251ff2cd86b